In [1]:
import sqlite3
import pandas as pd

df = pd.read_csv('./Data/cast.csv')
df = df.drop(columns=['row'])
conn = sqlite3.connect('database/final_data.db')


FileNotFoundError: [Errno 2] No such file or directory: './Data/cast.csv'

In [7]:
df_long = pd.melt(df, id_vars=['user_name'], var_name='cast_name', value_name='watch_count')
df_long = df_long[df_long['watch_count'] > 0]
print(df_long.head(100)) 
df_long.to_sql('user_director_counts', conn, if_exists='replace', index=False)
cast_names_set = set(df_long['cast_name'])
print(cast_names_set)

          user_name     cast_name  watch_count
0        deadalus22  Nicolas Cage           93
1         gayartkid  Nicolas Cage           16
2             tuoto  Nicolas Cage           31
5       eclectic_em  Nicolas Cage           22
10   thedayofthedot  Nicolas Cage           11
..              ...           ...          ...
341      demontrees  Nicolas Cage           34
343       jeffdlowe  Nicolas Cage           66
353     fuchsiadyke  Nicolas Cage           24
357         va24mar  Nicolas Cage           31
359        j1w_gtpl  Nicolas Cage           15

[100 rows x 3 columns]
{'Maya Rudolph', 'Kurt Russell', 'Harumi Sone', 'Jofre Soares', 'Daniel Brühl', 'Joe Chrest', 'Ikue Otani', 'Chishū Ryū', 'Joan Cusack', 'Toshio Furukawa', 'Bela Lugosi', 'Chevy Chase', 'Ching Miao', 'Michael Shannon', 'Moe Howard', 'Aubrey Plaza', 'Sylvester Stallone', 'Tatsuya Nakadai', 'George Sanders', 'Val Kilmer', 'Chris Rock', 'Danielle Harris', 'Ken Jeong', 'Kirsten Dunst', 'Eddie Marsan', 'Maaya Saka

In [8]:
cursor=conn.cursor()
cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS Cast (
    cast_id INTEGER PRIMARY KEY,
    cast_name TEXT NOT NULL UNIQUE
		);''')
for name in cast_names_set:
    cursor.execute('''
    INSERT OR IGNORE INTO Cast (cast_name)
    VALUES (?)
    ''', (name,))
conn.commit()
               
    

In [ ]:

def export_to_db(file_path, db_name, variable_name, value_name, main_table_name,foreign_table_name):
    df = pd.read_csv(file_path)
    if 'row' in df.columns:
        df = df.drop(columns=['row'])

    conn = sqlite3.connect(db_name)

    df_long = pd.melt(df, id_vars=['user_name'],
                      var_name=variable_name, value_name=value_name)
    df_long = df_long[df_long[value_name] > 0]

    df_long.to_sql(main_table_name, conn, if_exists='replace', index=False)

    cast_names_set = set(df_long[variable_name])

    cursor = conn.cursor()
    cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS {foreign_table_name} (
    {variable_name[:-3]}_id INTEGER PRIMARY KEY,
    {variable_name} TEXT NOT NULL UNIQUE
		);''')
    for name in cast_names_set:
        cursor.execute(f'''
		INSERT OR IGNORE INTO {foreign_table_name} ({variable_name})
		VALUES (?)
		''', (name,))
    conn.commit()
    conn.close()
  


In [ ]:

export_to_db('./Data/languages.csv', 'new_data.db', 'language', 'watch_count', 'Languages_count',"Language")
export_to_db('./Data/directors.csv', 'new_data.db', 'director_name', 'watch_count', 'Director_count',"Director")
export_to_db('./Data/countries.csv', 'new_data.db', 'country_name', 'watch_count', 'countries_count', 'Country')
export_to_db('./Data/countries_rating.csv', 'new_data.db', 'country_name', 'rating', 'country_rating', 'Country')


In [ ]:
export_to_db('./Data/cast.csv', 'new_data.db', 'cast_name', 'watch_count', 'Cast_count', 'Cast')
export_to_db('./Data/languages_rating.csv', 'new_data.db', 'language', 'rating', 'language_rating', 'Language')	
export_to_db('./Data/rating_by_decade.csv', 'new_data.db', 'decade', 'rating', 'decade_rating', 'Decade')	
export_to_db('./Data/rating_cast.csv', 'new_data.db', 'cast_name', 'rating', 'Cast_rating', 'Cast')	


In [ ]:
export_to_db('./Data/rating_directors.csv', 'new_data.db', 'director_name', 'rating', 'Director_rating', 'Director')	
export_to_db('./Data/watched_by_decade.csv', 'new_data.db', 'decade', 'watch_count', 'decade_count', 'Decade')	

Creating foreign keys for tables

In [ ]:
conn = sqlite3.connect("new_data.db")


def link_to_users(table_name, foreign_key, value,foreign_table_name):

    cursor=conn.cursor()
    cursor.executescript(f'''
              BEGIN TRANSACTION;   
              PRAGMA foreign_keys = ON;
              ALTER TABLE {table_name} ADD COLUMN user_id INTEGER;
							UPDATE {table_name}
							SET user_id = (
									SELECT user_id
									FROM User
									WHERE TRIM(User.user_name) = TRIM({table_name}.user_name)
							);
							ALTER TABLE {table_name} RENAME TO {table_name}_old;
							CREATE TABLE {table_name} (
							{foreign_key} INTEGER,
							user_id INTEGER,
							{value} INTEGER,
							user_name TEXT,
							FOREIGN KEY (user_id) REFERENCES User(user_id),
							FOREIGN KEY ({foreign_key}) REFERENCES {foreign_table_name}({foreign_key})
							);
							INSERT INTO {table_name} ( {foreign_key}, user_id, {value},user_name)
							SELECT  {foreign_key}, user_id, {value},user_name
							FROM {table_name}_old;
							''')
    


Changing user names to Userids in tables

In [ ]:
def user_name_to_user_id(table_name):
  cursor=conn.cursor()
  cursor.executescript(f'''
              begin transaction;
              ALTER TABLE {table_name} ADD COLUMN user_id INTEGER;
							UPDATE {table_name}
							SET user_id = (
									SELECT user_id
									FROM User
									WHERE TRIM(User.user_name) = TRIM({table_name}.user_name)
							);
       ''')

In [ ]:
user_name_to_user_id('decade_rating')

Creating unpivoted data for movie ratings

In [8]:
movie_rating_df=pd.read_csv('../Movies_rating.csv')
conn = sqlite3.connect('database/final_data.db')
movie_rating_df['User'] = movie_rating_df['User'].str.extract(r'https://www\.letterboxd\.com/([^/]+)/')
movie_rating_df['user_name']=movie_rating_df['User']
movie_rating_df.to_csv('./movies_rating2.csv', index=False)
movie_rating_df.drop(columns=['User'], inplace=True)
movie_rating_df=movie_rating_df.melt(id_vars=['user_name'], var_name='movie_name', value_name='rating')
movie_rating_df = movie_rating_df[movie_rating_df['rating'].notna()]
movie_rating_df = movie_rating_df[movie_rating_df['rating'] != 'No rating']
print(movie_rating_df.head(500))
movie_rating_df['title']=movie_rating_df['movie_name']
movie_rating_df.drop(columns=['movie_name'], inplace=True)
users_df = pd.read_sql("SELECT user_id, user_name FROM User", conn)
movies_df = pd.read_sql("SELECT movie_id, title FROM Movies", conn)
merged = movie_rating_df.merge(users_df, on="user_name", how="left")
merged = merged.merge(movies_df, on="title", how="left")
merged.drop(columns=['user_name', 'title'], inplace=True)
merged.to_csv('./movies_rating3.csv', index=False)

         user_name            movie_name rating
2    strawhatsanti  (500) Days of Summer    4.0
3       blakemower  (500) Days of Summer    4.0
4       skywvlkers  (500) Days of Summer    2.5
5       suspirliam  (500) Days of Summer    4.5
6            cobbb  (500) Days of Summer    4.0
..             ...                   ...    ...
922         yarjka  (500) Days of Summer    3.5
923        elizaez  (500) Days of Summer    4.0
925          jeaba  (500) Days of Summer    3.0
926           da94  (500) Days of Summer    3.5
938    calvinkemph  (500) Days of Summer    2.0

[500 rows x 3 columns]


Creating Database for Movies rating:

In [19]:
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS Movies_ratings (
    user_id INTEGER,
    movie_id INTEGER,
    rating REAL,
    FOREIGN KEY (user_id) REFERENCES User(user_id),
    FOREIGN KEY (movie_id) REFERENCES Movies(movie_id)
);
""")
merged = merged[merged['user_id'].notnull()]
merged.dropna(inplace=True)
merged.to_sql("Movies_ratings", conn, if_exists="append", index=False)

469082

In [18]:
conn.execute("DROP TABLE Movies_ratings")